In [1]:
import os
try:
  !pip install groq
  from groq import Groq
except:
  pass
from google.colab import userdata

# Probando Groq

In [2]:
try:
  #groq_api_key = userdata.get("GROQ_API_KEY")
  groq_api_key = "aqui va el api key"
except Exception as e:
  print(f"Error al cargar la clave de Groq {e}")

In [3]:
client = Groq(
    api_key=groq_api_key
)

In [4]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"user",
            "content":"Explica el mecanismo de atencion en los transformers"
        }
    ],
    model="llama3-70b-8192"
)
print(chat_completion.choices[0].message.content)

Uno de los conceptos más interesantes y poderosos en la arquitectura de los Transformers: la atención (Attention Mechanism)!

La atención es un mecanismo clave en la arquitectura de los Transformers, introducida originalmente por Vaswani et al. en su paper "Attention is All You Need" (2017). Este mecanismo permite que el modelo se centre en las partes más relevantes de la entrada mientras procesa la información, lo que mejora significativamente su capacidad para capturar relaciones entre diferentes partes de la entrada.

**Cómo funciona la atención en los Transformers**

En un Transformer, la atención se aplica en la capa de " self-attention" (atención a sí mismo), que es una parte crítica del modelo. La atención se utiliza para calcular la representación de cada posición en la secuencia de entrada, ponderando la importancia de cada una de las otras posiciones.

El mecanismo de atención se puede descomponer en tres pasos:

1. **Crear matrices de queries (Q), keys (K) y values (V)**: se

# Vamos a empezar con la estructura del RAG

## Retrieval

In [5]:
try:
  !pip install langchain_community
  !pip install pypdf
except:
  pass

### text splitter

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=500,
    add_start_index=True,
)

In [8]:
pdf_path = "/content/gauss_fields.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"Documento cargado, numero de paginas {len(documents)}")

Documento cargado, numero de paginas 19


In [9]:
loader

In [10]:
chunks = text_splitter.split_documents(documents)

In [11]:
print(f"Número de chunks: {len(chunks)}")
print("\n Primer chunk:")
print(chunks[0].page_content)

Número de chunks: 19

 Primer chunk:
arXiv:2006.11870v1  [math.NT]  21 Jun 2020
GENUS FIELDS OF KUMMER ℓn–CYCLIC EXTENSIONS
CARLOS DANIEL REYES–MORALES AND GABRIEL VILLA–SAL V ADOR
A B S T R A C T . W e give a construction of the genus ﬁeld for Kummer ℓn-cyclic ex-
tensions of rational congruence function ﬁelds, where ℓ is a prime number . First,
we compute the genus ﬁeld of a ﬁeld contained in a cyclotomic f unction ﬁeld, and
then for the general case. This generalizes the result obtai ned by Peng for a Kum-
mer ℓ-cyclic extension. Finally , we study the extension (K1K2)ge/(K1)ge(K2)ge,
for K1, K2 abelian extensions of k.
1. I NT RODU CT ION
The origin of genus ﬁelds dates back to C. F . Gauss [8] in his wo rk about binary
quadratic forms. For a ﬁnite ﬁeld extension K of Q, the rational number ﬁeld, the
genus ﬁeld Kge is deﬁned as the maximum unramiﬁed ﬁeld extension of K such
that it is the composite Kk∗, where k∗ is an abelian ﬁeld extension over Q. This
deﬁnition is due to A. Fr ¨ 

### Embedding
embedding model from HungginFace

In [12]:
try:
  !pip install -qU langchain-huggingface
except:
  pass

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
vector_1 = embeddings.embed_query(chunks[0].page_content)
vector_2 = embeddings.embed_query(chunks[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 384

[-0.09074479341506958, 0.06299955397844315, 0.06687162071466446, -0.02946328930556774, -0.03094075806438923, 0.011214579455554485, 0.039026226848363876, 0.03662385419011116, 0.05328264832496643, -0.04078393429517746]


### Vector stores

In [15]:
try:
  !pip install faiss-cpu
except:
  pass

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [16]:
embedding_dim = len(embeddings.embed_query("Hola Mundo"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(), # los documentos se almacenan en la RAM de la pc
    index_to_docstore_id={}
)

In [17]:
ids = vector_store.add_documents(documents=chunks)

In [18]:
ids

['805e27d4-2888-45e5-bbd0-b620182db836',
 '5979c5ae-4b4c-4b9a-a944-845b4b1ec80e',
 '56c241d7-c688-4d52-9637-d107ef0c7a30',
 '8d62cd24-ec39-4f95-8a03-660177a81e06',
 'c8e478cd-bc35-4bcf-a625-0456387d7226',
 '9f2c6d47-f004-4ac1-8564-b329de1ad33f',
 '9adabc3f-6203-4854-a40e-afc8c0d87787',
 'c5c281de-8009-4bee-a067-17fa8993a5d0',
 '96df238e-8da1-4540-b5c5-173d03c19920',
 'f8901eca-a66f-48ec-881a-714e1a30f097',
 'c0b688ef-7204-42c3-a240-d1f583ce2758',
 'e747f59e-ea62-4a95-b1b6-dba1141329f4',
 '801150fe-6936-4f76-abd1-a00462f6d8cd',
 '392c276a-23e8-4320-83fc-aae622272d96',
 'c06b2963-42fd-4e52-90b7-bdc4e1f3a3c1',
 '03a5c421-c277-4b7c-88d2-9ab57e5d7b4d',
 '518812f3-2fb4-4c45-9afd-96d31b32c989',
 '1bdd9beb-887f-481c-a84b-ac36f0f675d9',
 '07922309-9bae-4838-9367-3d4100d92e28']

In [19]:
results = vector_store.similarity_search(
    "genus field"
)

print(results[0])

page_content='GENUS FIELDS OF KUMMER ℓn–CYCLIC EXTENSIONS 19
DE PA RTA M E N T O D E CO N T R O L A U T O M ´A T IC O , C E N T R O D E IN V E S T I G A C I ´O N Y D E E S T U D I O S AVA N-
Z A D O S D E L I. P . N.
E-mail address : mcenigm@gmail.com, creyes@ctrl.cinvestav.mx
DE PA RTA M E N T O D E CO N T R O L A U T O M ´A T IC O , C E N T R O D E IN V E S T I G A C I ´O N Y D E E S T U D I O S AVA N-
Z A D O S D E L I. P . N.
E-mail address : gvillasalvador@gmail.com, gvilla@ctrl.cinvestav.mx' metadata={'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2021-11-08T03:55:40-05:00', 'moddate': '2021-11-08T03:55:40-05:00', 'title': '', 'subject': '', 'author': '', 'keywords': '', 'source': '/content/gauss_fields.pdf', 'total_pages': 19, 'page': 18, 'page_label': '19', 'start_index': 0}


In [20]:
results = await vector_store.asimilarity_search("What does the proposition 2.1. say?")
print(results[0])

page_content='14 C. REYES AND G. VILLA
Proposition 5.3. Let K1, K2 ⊆ nk(Λ N )m, K = K1K2 and E1, E2 as before. If Kge =
(K1)ge(K2)ge, then Ege = ( E1)ge(E2)ge.
Proof. Let K = K1K2 ⊆ nk(Λ N )m and E := E1E2 ⊆ k(Λ N ). Consider the ﬁelds
(Kge)m and (Ege)m. Now , from [4, Theorem 2.2] we have (Ege)m = ( Kge)m and
((Ki)ge)m = (( Ei)ge)m, i = 1 , 2. Therefore
(Ege)m = ( Kge)m = (( K1)ge(K2)ge)m = (( K1)ge)m((K2)ge)m
= (( E1)ge)m((E2)ge)m = (( E1)ge(E2)ge)m.
Finally , by the Galois correspondence, it follows
Ege = ( Ege)m ∩ k(Λ N ) = (( E1)ge(E2)ge)m ∩ k(Λ N ) = ( E1)ge(E2)ge.
□
The converse of Proposition 5.2 does not hold in general, tha t is, if Ege =
(E1)ge(E2)ge, then the equality Kge = ( K1)ge(K2)ge may fail.
Example 5.4. Let K1 = k( ℓ√P1), K2 = k( ℓ√γP2) and K = K1K2 be such that
P1, P2 ∈ R+
T are different polynomials with deg P1 = a, 1 ≤ a < ℓ , deg P2 = ℓ − a
and γ /∈ (F∗
q )ℓ. Then we have Ei = k( ℓ
√
(Pi)∗), i = 1 , 2 and E = E1E2. If χPi is the
Dirichlet character associated wit

## Retrieval & Generations

integracion del mecanismo de recuperacion con Groq

In [21]:
try:
  !pip install -qU langchain-groq
except:
  pass

In [22]:
os.environ["GROQ_API_KEY"] = groq_api_key

In [23]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0.8,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

### creando el retriever

In [24]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5}) # recuperar los 5 vectores más similares o en otras palabras los 3 documentos más relevantes

### plantilla para el prompt

In [25]:
from langchain_core.prompts import ChatPromptTemplate

In [26]:
template = """
Eres un asistente de investigacion inteligente y traductor simultaneo. Tu tarea es responder a las preguntas del usuario basandote estrictamente en el 'Contexto' proporcionado.

Instrucciones:
1. Lee cuidadosamente la 'pregunta del usuario' y el 'contexto' que se te proporciona. Ambos estaran en ingles.
2. Utiliza únicamente la información que encuentres en el 'Contexto' para formular tu respuesta. No uses conocimiento externo.
3. Si la respuesta puede encontrarse en el 'Contexto', formula una respuesta clara y concisa en **ESPAÑOL**.
4. Si la información necesaria para responder la pregunta no está presente en el 'Contexto', simplemente indica en español: "Lo siento, no tengo suficiente información en los documentos proporcionados para responder a esa pregunta.".
5. Evita inventar información o hacer suposiciones.
6. Manten tus respuestas objetivas y neutrales.

Contexto:
{context}

Pregunta:
{question}

Respuesta:
"""

prompt = ChatPromptTemplate.from_template(template)

In [27]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nEres un asistente de investigacion inteligente y traductor simultaneo. Tu tarea es responder a las preguntas del usuario basandote estrictamente en el \'Contexto\' proporcionado.\n\nInstrucciones:\n1. Lee cuidadosamente la \'pregunta del usuario\' y el \'contexto\' que se te proporciona. Ambos estaran en ingles.\n2. Utiliza únicamente la información que encuentres en el \'Contexto\' para formular tu respuesta. No uses conocimiento externo.\n3. Si la respuesta puede encontrarse en el \'Contexto\', formula una respuesta clara y concisa en **ESPAÑOL**.\n4. Si la información necesaria para responder la pregunta no está presente en el \'Contexto\', simplemente indica en español: "Lo siento, no tengo suficiente información en los documentos proporc

aqui vamos a conectar todas las piezas

In [28]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {'context': retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# echando a andar el RAG

In [29]:
consulta = "Desde donde viene el origen del Genus Field?"
respuesta = rag_chain.invoke(consulta)
print(f"Pregunta: {consulta}")
print(f"Respuesta: {respuesta}")

Pregunta: Desde donde viene el origen del Genus Field?
Respuesta: Según el contexto, el origen del Genus Field se remonta a la obra de C. F. Gauss sobre formas cuadradas binarias.


In [30]:
consulta_2 = "De qué trata el documento Genus field of kummer?"
respuesta_2 = rag_chain.invoke(consulta_2)
print(f"Pregunta: {consulta_2}")
print(f"Respuesta: {respuesta_2}")

Pregunta: De qué trata el documento Genus field of kummer?
Respuesta: El documento "Genus Fields of Kummer ℓn–Cyclic Extensions" trata sobre la construcción del campo de género para extensiones cíclicas de Kummer de campos de congruencia funcionales, donde ℓ es un número primo. El documento proporciona una definición de campo de género para campos de congruencia funcionales y estudia sus propiedades, incluyendo la relación entre el campo de género y el campo de clase de Hilbert. También se presentan resultados sobre la estructura del campo de género para extensiones cíclicas de Kummer y se comparan con resultados previos en la literatura.


In [32]:
consulta_3 = "Cual es la documentación que se esta usando en el documento?"
respuesta_3 = rag_chain.invoke(consulta_3)
print(f"Pregunta: {consulta_3}")
print(f"Respuesta: {respuesta_3}")

Pregunta: Cual es la documentación que se esta usando en el documento?
Respuesta: La documentación que se está usando en el documento se refiere a la lista de referencias bibliográficas etiquetadas con números del 1 al 20. Estas referencias incluyen artículos de investigación, libros y otros trabajos académicos en el campo de las matemáticas, específicamente en teoría de números y álgebra.


In [33]:
consulta_4 = "en el documento qué es lo que dice la proposition2.1."
respuesta_4 = rag_chain.invoke(consulta_4)
print(f"Pregunta: {consulta_4}")
print(f"Respuesta: {respuesta_4}")

Pregunta: en el documento qué es lo que dice la proposition2.1.
Respuesta: Lo siento, no tengo suficiente información en los documentos proporcionados para responder a esa pregunta. La Proposición 2.1 no se encuentra en los documentos proporcionados.


In [34]:
consulta_5 = "what does the Notation say us?"
respuesta_5 = rag_chain.invoke(consulta_5)
print(f"Pregunta: {consulta_5}")
print(f"Respuesta: {respuesta_5}")

Pregunta: what does the Notation say us?
Respuesta: According to the Contexto, the Notation section provides a list of definitions for various mathematical symbols and expressions used throughout the paper. Specifically, it defines the following:

* gcd: the greatest common divisor
* lcm: the least common multiple
* νℓ: the valuation with respect to ℓ, a prime number
* k: the rational function field Fq(T)
* RT: the polynomial ring Fq[T]
* R+: the set of monic and irreducible polynomials in RT
* S∞(K): the set of primes in K over P∞
* ne∞(E/F): the ramification index of P∞ in the field extension E/F
* nf∞(E/F): the inertia degree of P∞ in the field extension E/F
* ΛN: the N-torsion of the Carlitz's module for N ∈ RT \ {0}
* Kge: the genus field of K
* Kgex: the extended or narrow genus field of K
* k(ΛN): the cyclotomic function field over k determined by N
* k(ΛN)+: the maximum real subfield of k(ΛN)
* Ln: the subfield of k(ΛT−n−1) fixed by F∗q
* Km: the field KFqm
* nK: the field KLn


In [35]:
consulta_6 = "what is the origin of genus field?"
respuesta_6 = rag_chain.invoke(consulta_6)
print(f"Pregunta: {consulta_6}")
print(f"Respuesta: {respuesta_6}")

Pregunta: what is the origin of genus field?
Respuesta: De acuerdo con el contexto, el origen de los campos de género se remonta al trabajo de C. F. Gauss sobre formas cuadráticas binarias.


In [36]:
consulta_7 = "what does the M.Rosen's definition say?"
respuesta_7 = rag_chain.invoke(consulta_7)
print(f"Pregunta: {consulta_7}")
print(f"Respuesta: {respuesta_7}")

Pregunta: what does the M.Rosen's definition say?
Respuesta: Según el contexto, la definición de M. Rosen dice que el campo de clase de Hilbert (relativo a un conjunto S de lugares de un campo de función global K) es el máximo campo abeliano no ramificado de K tal que los lugares en S se descomponen completamente en él.


# Una pequeña prueba con gradio

In [37]:
!pip install gradio
import gradio as gr

In [38]:
def rag_chat(consulta):
  return rag_chain.invoke(consulta)

In [39]:
interfaz = gr.Interface(fn=rag_chat, inputs="text", outputs="text")
interfaz.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0c2f98310a2fb45846.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
